### Spark 설정하기

In [10]:
# [+] SparkConf, SparkContext 임포트
from pyspark import SparkConf, SparkContext

In [11]:
"""
    [+] SparkConf 객체 생성 및 설정
    - setMaster(), 마스터 노드 위치: "local"
    - 앱이름: "transformations_actions"
"""

conf = SparkConf().setMaster('local').setAppName('transformation_actions')

In [12]:
# [+] SparkContext 객체 생성
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=transformation_actions, master=local) created by __init__ at <ipython-input-3-c9cee7789c39>:2 

In [13]:
# Spark 전체 설정 값 확인
sc.getConf().getAll()

# getConf() : SparkConf()객체인 conf가 return
# getAll() : Spakr에 설정된 모든 값 확인 가능

[('spark.master', 'local'),
 ('spark.driver.host', 'LAPTOP-K5S3UUQF'),
 ('spark.driver.port', '62959'),
 ('spark.app.id', 'local-1654695735930'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1654695732413'),
 ('spark.app.name', 'transformation_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [14]:
# SpackContext 객체 하나 더 생성하기(과연?)
sc = SparkContext(conf=conf)
# 오류 발생함 -> Cannot run multiple SparkContexts at once (여러 개의 spark context설정할 수 없다)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=transformation_actions, master=local) created by __init__ at <ipython-input-3-c9cee7789c39>:2 

In [15]:
# SparkContext는 프로그램 내에서 하나의 객체만 생성할 수 있다. (= Singleton 패턴)
sc.stop()       # SparkContext 객체 해제하기

In [16]:
# 다시 생성하기
sc = SparkContext(conf=conf)

### 기본 Actions 함수 사용하기

In [17]:
"""
    [+] List로부터 RDD 객체 생성
    - SparkContext.parallelize()
    - 리스트 값: 
    "짜장면", "마라탕", "짬뽕", 
    "떡볶이", "쌀국수", "짬뽕", 
    "짜장면", "짜장면", "짜장면", 
    "라면", "우동", "라면"
"""
lst = ["짜장면", "마라탕", "짬뽕", 
    "떡볶이", "쌀국수", "짬뽕", 
    "짜장면", "짜장면", "짜장면", 
    "라면", "우동", "라면"]

foods = sc.parallelize(lst)

In [18]:
# [+] foods RDD 내용 출력
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [19]:
# [+] foods RDD 원소 값 별로 카운팅
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [20]:
# take(n): 처음 n개의 원소를 출력
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [21]:
foods.take(5)

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수']

In [22]:
# first(): 첫 번째 값을 출력
foods.first()

'짜장면'

In [23]:
# count(): 원소 갯수 출력(중복된 원소 포함)
foods.count()

12

In [24]:
"""
    distinct(): unique한 원소로 구성된 RDD 생성
    - distinct() 자체는 transformation 이므로, collect()와 같은 action 을 결합하여 사용
"""

foods.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [25]:
""" 
    foreach(): RDD 원소를 하나씩 꺼내서 함수 연산을 적용
    - Worker node 에서 실행되기 때문에 Driver Program 에서 결과를 볼 수 없음
    - Worker node 에서 따로 실행할 함수(예: 로깅)들을 적용할 때 사용
"""

foods.foreach(lambda x: print(x)) 
# print 보이지 않음.

### 기본 Transformations 함수 사용하기

In [26]:
# Transformations 은 지연 실행된다.
sc.parallelize([1, 2, 3]).map(lambda x: x + 2)

PythonRDD[13] at RDD at PythonRDD.scala:53

In [27]:
# Actions 를 만나면 즉시 실행된다.
sc.parallelize([1, 2, 3]).map(lambda x: x + 2).collect()

[3, 4, 5]

In [28]:
sc.parallelize([1, 2, 3]).map(lambda x: x * 2).collect()

[2, 4, 6]

In [29]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [30]:
# [+] 리스트(movies) -> RDD 객체(moviesRDD) 생성
moviesRDD = sc.parallelize(movies)

In [31]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [33]:
# 단어 단위로 쪼개서 보기
# map :  원소 개수 유지(one-to-one) -> movieRDD에 적용하면 ['그린','북'] 이런 식으로 나올 것
# flatMap : 원소 개수 증가(one-to-many transformation)
flatMovies = moviesRDD.flatMap(lambda x: x.split(" "))

In [34]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [35]:
"""
    [+] flatMovies 에서 '매트릭스'를 제외
    - filter(lambda ...)
    - 저장할 객체: filteredMovies
"""

filteredMovies = flatMovies.filter(lambda x: x != '매트릭스')

In [36]:
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [37]:
"""
    집합 관련 transformations 연산
"""

num1 = sc.parallelize([1, 2, 3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [38]:
# 교집합(intersection) 구하기 -> num1기준으로 num2
num1.intersection(num2).collect()

[4]

In [39]:
# 합집합(union) 구하기
num1.union(num2).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [40]:
# 차집합(substract) 구하기 -> 순서 바뀌어서 출력될 수 있음
num1.subtract(num2).collect()

[2, 1, 3]

In [41]:
numUnion = num1.union(num2)

In [42]:
numUnion.collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [43]:
"""
    sample(withReplacement, fraction): 원소를 랜덤 샘플링하여 새로운 RDD를 생성하는 narrow transformation
    - withReplacement, bool[optional]: 중복 샘플링 허용 여부
    - fraction, float[optional]: 샘플링 확률[0~1], 확률이므로 정확한 사이즈가 보장되지 않음(10개 원소 있다고 꼭 5개 나오는 게 아니라 각각의 원소에 대한 확률임)
    - seed: 결과 재현(reproducibility)을 위한 설정 값
"""

numUnion.sample(True, 0.5).collect()

[1, 1, 7, 7, 7, 10]

In [44]:
# seed 파라미터를 설정하면 동일한 결과를 반복적으로 얻을 수 있다.
numUnion.sample(True, .5, seed=5).collect()

[1, 4, 6, 9, 9, 10]

In [45]:
"""
    groupBy(): 특정 기준(함수로 정의)으로 그룹핑을 수행하는 wide transformation
"""

foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", 
                        "떡볶이", "쌀국수", "짬뽕", 
                        "짜장면", "짜장면", "짜장면",  
                        "라면", "우동", "라면", 
                        "치킨", "돈까스", "회", 
                        "햄버거", "피자"])

In [46]:
#  원소의 첫 번째 값을 기준으로 그룹핑
foodsGroup = foods.groupBy(lambda x: x[0])

In [47]:
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x2929cda6340>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x2929cda6f40>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x2929cda6e50>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x2929ce72d00>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x2929ce72070>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x2929ce72280>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x2929ce72a30>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x2929ce729a0>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x2929ce72a90>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x2929ce7d100>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x2929ce7d670>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x2929ce7de50>)]

In [48]:
# collect() 결과를 리스트 객체(res)에 저장
res = foodsGroup.collect()

In [49]:
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x2929ce7d9a0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x2928d5a5be0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x2928d5a5100>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x2929cea5220>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x2929cea50a0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x2929cea5c10>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x2929cea5190>),
 ('치', <pyspark.resultiterable.ResultIterable at 0x2929cea53d0>),
 ('돈', <pyspark.resultiterable.ResultIterable at 0x2929cea5460>),
 ('회', <pyspark.resultiterable.ResultIterable at 0x2929cea5400>),
 ('햄', <pyspark.resultiterable.ResultIterable at 0x2929cea5610>),
 ('피', <pyspark.resultiterable.ResultIterable at 0x2929cea5e20>)]

In [51]:
# ResultIterable 객체는 list로 쉽게 변환 가능하다.
list(res[0][1]) # resultiterable가져온 것

['짜장면', '짜장면', '짜장면', '짜장면']

In [52]:
# for 문을 이용하여 구조적으로 결과를 출력
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [53]:
# groupBy()는 모드 연산과 같이 수학 연산에도 사용될 수 있다.
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [54]:
nums.groupBy(lambda x: x % 2).collect()

[(1, <pyspark.resultiterable.ResultIterable at 0x2929ceadd30>),
 (0, <pyspark.resultiterable.ResultIterable at 0x2929ce7ddf0>)]

In [52]:
# [+] '1' 그룹에 해당되는 원소 리스트 출력하기
list(nums.groupBy(lambda x: x % 2).collect()[0][1])

[1, 3, 5, 7, 9]

In [53]:
# [+] '0' 그룹에 해당되는 원소 리스트 출력하기
list(nums.groupBy(lambda x: x % 2).collect()[1][1])

[2, 4, 6, 8, 10]